In [1]:
from transformers import BertModel,BertTokenizer
import torch
import pandas as pd
import numpy as np


c:\Users\Anwender\AppData\Local\anaconda3\envs\pytorch-gpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
!pip install kagglehub   

Defaulting to user installation because normal site-packages is not writeable


In [8]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
deepcontractor_200k_short_texts_for_humor_detection_path = kagglehub.dataset_download('deepcontractor/200k-short-texts-for-humor-detection')

print('Data source import complete.')


ModuleNotFoundError: No module named 'kagglehub'

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
df=pd.read_csv("/kaggle/input/200k-short-texts-for-humor-detection/dataset.csv")

In [ ]:
#checking the dataset
df.tail(55000)

The Dataset is large.so what i am doing here is slicing it down for quicker traning time

using 40000 samples

In [ ]:
# Separate the DataFrame into two based on the 'humor' column
df_humor = df[df['humor'] == 1]
df_not_humor = df[df['humor'] == 0]

# Define the desired number of samples for each class
desired_samples_per_class = 20000

# Check if there are enough samples in each class
if len(df_humor) < desired_samples_per_class or len(df_not_humor) < desired_samples_per_class:
    print(f"Warning: Not enough samples in one or both classes to get {desired_samples_per_class} samples each.")
    print(f"Available samples in 'humor=1': {len(df_humor)}")
    print(f"Available samples in 'humor=0': {len(df_not_humor)}")


# Randomly sample the desired number of instances from each class
df_humor_balanced = df_humor.sample(n=desired_samples_per_class, random_state=42)
df_not_humor_balanced = df_not_humor.sample(n=desired_samples_per_class, random_state=42)

# Concatenate the balanced DataFrames back together
df_balanced = pd.concat([df_humor_balanced, df_not_humor_balanced])

# Shuffle the balanced DataFrame to mix the classes
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Now df_balanced is a DataFrame with the specified number of samples for each class
print(f"Original DataFrame shape: {df.shape}")
print(f"Balanced DataFrame shape: {df_balanced.shape}")
print(f"Value counts in balanced DataFrame 'humor' column:\n{df_balanced['humor'].value_counts()}")

Importing models

In [ ]:
# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model_name = "bert-base-uncased"
tokenzier = BertTokenizer.from_pretrained(model_name)
model=BertModel.from_pretrained(model_name)
model.eval()

Creating embadding and using Tqdm tracking the progress

In [ ]:
from tqdm import tqdm

# Enable tqdm for pandas apply
tqdm.pandas()

# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the model and tokenizer
model_name = "bert-base-uncased"
tokenzier = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Move the model to the chosen device (GPU if available, otherwise CPU)
model.to(device)
model.eval()

embeddings = []

# Load the cudf.pandas extension if running on GPU
if device.type == 'cuda':
    try:
        %load_ext cudf.pandas
        print("cuDF pandas extension loaded.")
    except ImportError:
        print("cuDF not installed. Pandas operations will run on CPU.")

def get_embedding (text):
  # Move the input tensors to the same device as the model
  inputs = tokenzier(text,
                     return_tensors="pt",truncation=True,padding=True,max_length=64).to(device)
  with torch.no_grad():
    outputs = model(**inputs)
    # Move the output tensor back to CPU before converting to numpy
    cls_embedding = outputs.last_hidden_state[:,0,:].cpu().numpy()
    embeddings.append(cls_embedding.squeeze())
    return cls_embedding.squeeze()

 #apply to dataframe
df_balanced['embedding'] = df_balanced['text'].progress_apply(get_embedding)

# Example: saving embeddings
import numpy as np
np.save("bert_embeddings.npy", embeddings)

In [ ]:
#df_balanced.to_csv("bert_embeddings.csv",index=False)

In [ ]:
df_balanced.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

X=np.stack(df_balanced['embedding'].values)
y=df_balanced['humor']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
ls_model = LogisticRegression(max_iter=50000)
ls_model.fit(X_train,y_train)
y_pred =ls_model.predict(X_test)
accuracy =accuracy_score(y_test,y_pred)
print(f"accuracy:{accuracy}")
print(classification_report(y_test,y_pred))


we got Accuracy of 96%

let's Test the model with new data which we did not used

In [ ]:
from operator import mod


def predict_text_class(text):
    # Step 1: Tokenize and get [CLS] embedding
    inputs = tokenzier(text, return_tensors="pt", truncation=True, padding=True, max_length=64)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()

    # Step 2: Predict using trained classifier
    prediction = ls_model.predict([cls_embedding])[0]
    proba = ls_model.predict_proba([cls_embedding])[0]

    return prediction, proba


In [ ]:
text = "The introverted parent's guide to surviving the holidays"
label, probabilities = predict_text_class(text)

print(f"Predicted class: {label}")
print(f"Probabilities: {probabilities}")

90% time model is predicting correct answer

In [ ]:
pd.set_option('display.max_colwidth', None)
print(df.iloc[89000])

trying Deep learning model to check if we can get better accuracy

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

# MLP model
mlp = MLPClassifier(
    hidden_layer_sizes=(512, 256),  # Two hidden layers
    activation='relu',
    solver='adam',
    batch_size=32,
    max_iter=100,
    random_state=42,
    verbose=True
)

# Train the model
mlp.fit(X_train, y_train)

# Predict on test set
y_pred = mlp.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))